# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298521744663                   -0.85    5.2
  2   -8.300208609694       -2.77       -1.26    1.0
  3   -8.300435874183       -3.64       -1.89    2.4
  4   -8.300461546804       -4.59       -2.75    2.9
  5   -8.300464239508       -5.57       -3.14    2.5
  6   -8.300464434425       -6.71       -3.33    4.5
  7   -8.300464537401       -6.99       -3.48    1.5
  8   -8.300464596541       -7.23       -3.62    9.4
  9   -8.300464628348       -7.50       -3.80    1.6
 10   -8.300464639682       -7.95       -3.99    1.8
 11   -8.300464643637       -8.40       -4.30    6.9


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64194105072                   -0.70    4.8
  2   -16.67832241446       -1.44       -1.14    1.4
  3   -16.67920326230       -3.06       -1.88    4.4
  4   -16.67927824478       -4.13       -2.74    2.0
  5   -16.67928585332       -5.12       -3.08    5.8
  6   -16.67928620084       -6.46       -3.49    2.2
  7   -16.67928621977       -7.72       -3.95    1.9
  8   -16.67928622409       -8.36       -4.54    2.5


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32511515474                   -0.56    6.6
  2   -33.33263185443       -2.12       -1.00    1.2
  3   -33.33411013230       -2.83       -1.74    4.0
  4   -33.33428094386       -3.77       -2.64    3.1
  5   -33.33588931383       -2.79       -2.41    4.4
  6   -33.33694239053       -2.98       -2.53   10.2
  7   -33.33694258402       -6.71       -2.54    1.0
  8   -33.33683335375   +   -3.96       -2.30    1.6
  9   -33.33680554400   +   -4.56       -2.26    1.0
 10   -33.33682188181       -4.79       -2.29    1.0
 11   -33.33684592509       -4.62       -2.34    1.0
 12   -33.33685219546       -5.20       -2.35    1.0
 13   -33.33687453116       -4.65       -2.41    1.0
 14   -33.33688543761       -4.96       -2.45    1.0
 15   -33.33688603528       -6.22       -2.45    1.0
 16   -33.33691092777       -4.60       -2.57    1.0
 17   -33.33694374213       -4.48       -3.79 

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298217012355                   -0.85    5.4
  2   -8.300239686384       -2.69       -1.59    1.0
  3   -8.300434845964       -3.71       -2.64    2.2
  4   -8.300431356349   +   -5.46       -2.50    6.0
  5   -8.300463264162       -4.50       -3.19    1.0
  6   -8.300464577849       -5.88       -3.74    2.4
  7   -8.300464641274       -7.20       -4.50    1.6


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31636514318                   -0.56    6.8
  2   -33.29713838103   +   -1.72       -1.26    1.8
  3   -4.482425310431   +    1.46       -0.34    6.2
  4   -33.33067012475        1.46       -1.74    5.6
  5   -33.23382010246   +   -1.01       -1.33    3.8
  6   -33.11421466932   +   -0.92       -1.39    3.5
  7   -33.02408973747   +   -1.05       -1.31    4.9
  8   -33.33600744603       -0.51       -2.34    3.9
  9   -33.33512327124   +   -3.05       -2.21    3.0
 10   -33.33639621147       -2.90       -2.44    2.4
 11   -33.33690398118       -3.29       -3.04    2.0
 12   -33.33689344834   +   -4.98       -3.16    3.0
 13   -33.33694331829       -4.30       -3.69    2.5
 14   -33.33694322494   +   -7.03       -4.03    2.8


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.